# **Practico Mentoria - Aprendizaje No Supervisado**

Se propone la elaboración de un informe o presentación, en formato estatico:
* PDF
* Markdowns
* Google Docs

Que responda a las cuestiones solicitadas en cada seccion de esta **Jupyter Notebook**.

La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase.  
Por lo cual debe estar redactado de forma consisa y comprensible.

## **Referencias**:
* https://towardsdatascience.com/how-to-apply-k-means-clustering-to-time-series-data-28d04a8f7da3
* https://www.intechopen.com/online-first/clustering-of-time-series-data
* http://halweb.uc3m.es/esp/Personal/personas/amalonso/esp/ASDM-C02-clustering.pdf
* https://tslearn.readthedocs.io/en/stable/

## **Clustering to Time Series Data**

La idea de este practico es poder aplicar Tecnicas de Clustering sobre los diferentes `PUNTO_MEDICION` (que podemos considerar como Series Temporales independientes), con el fin de determinar diferentes agrupamientos en donde los `PUNTOS_MEDICION` sean similares, en algun sentido. 

Para ello pueden utilizar la libreria `tslearn`, la cual es una libreria de machine learning para el analisis de Series Temporales, construidas a partir de la las librerias `scikit-learn` y `numpy` entre otras.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(15, 5)})

In [ ]:
filename = "../Introduccion_Mentoria/dataset_inf_telec_20200501T130000_20200727T010000_v1.csv"

In [ ]:
df = pd.read_csv(
    filename,
    dtype={
        'ID_EQUIPO': int,
        'PUNTO_MEDICION': str,
        'CAPACIDAD_MAXIMA_GBS': float,
        'PASO': int,
        'LATENCIA_MS': float,
        'PORCENTAJE_PACK_LOSS': float,
        'INBOUND_BITS': np.float64,
        'OUTBOUND_BITS': np.float64,
        'MEDIDA': str,
    },
    index_col='FECHA_HORA',
    parse_dates=[
        'FECHA_INICIO_MEDICION',
        'FECHA_HORA',
        'FECHA_FIN_MEDICION',
    ],
    na_values=['NaN']
)

print(df.shape)
display(df.head(10))